In [1]:
!pip install implicit
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import CosineRecommender, BM25Recommender, TFIDFRecommender

     |████████████████████████████████| 1.1MB 5.2MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp37-cp37m-linux_x86_64.whl size=3406380 sha256=90abc0ef702fa2b58bae45cda85a89f3bc89417ca0732da7c5a76389d69d2783
  Stored in directory: /root/.cache/pip/wheels/bf/d4/ec/fd4f622fcbefb7521f149905295b2c26adecb23af38aa28217
Successfully built implicit


In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from tqdm.auto import tqdm

from itertools import islice, cycle
from more_itertools import pairwise
import scipy.sparse as sp
import datetime

from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [3]:
top_N = 10

#Load Data

In [4]:
all_data = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/MTS_teta/Kaggle_recomend_sys/all_data.pkl')
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MTS_teta/Kaggle_recomend_sys/sample_submission.csv')
df_items = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/MTS_teta/Kaggle_recomend_sys/items_preprocessed.pickle')
df_users = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/MTS_teta/Kaggle_recomend_sys/users_preprocessed.pickle')
df = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/MTS_teta/Kaggle_recomend_sys/interactions_preprocessed.pickle')

In [ ]:
sub_user_data = pd.merge(submission, df_users, how='left', left_on='Id', right_on='user_id')
print(sub_user_data['sex'].isna().sum(), sub_user_data['age'].isna().sum())

580 547


In [ ]:
all_data.head(3)

,user_id,item_id,progress,rating,start_date,age,sex,year,authors_code,auhurs_genres
0,90133,82910.0,100.0,NaN,2018-01-01,55_64,0,2015,1,современные детективы галина романова
1,159130,331068.0,70.0,NaN,2018-01-01,35_44,0,2015,1,героическое фэнтези боевое фэнтези алексей пехов
2,80061,26540.0,69.0,4.0,2018-01-01,25_34,0,2016,1,зарубежные детективы современные детективы миш...


In [ ]:
#Заменим пропущенный рейтинг на 4,5 там где книгу дочитали и на 3,5 где недочитали не хочу терять столько данных, 
#потом лучше уберу непопулярные книги. а пока оставим только закодирую отутствие оценки в отдельный столбец
all_d = all_data.copy(deep = True)
all_d.loc[(all_d.progress >= 90), 'rating'] = all_d.loc[(all_d.progress >= 90), 'rating'].fillna(4.5)
all_d.loc[(all_d.progress < 90), 'rating'] = all_d.loc[(all_d.progress < 90),'rating'].fillna(3.5)
all_d['rating_code'] = all_d.rating
all_d.loc[(all_d.rating_code != 3.5) & (all_d.rating_code != 4.5), 'rating_code'] = 1
all_d.loc[(all_d.rating_code == 3.5) | (all_d.rating_code == 4.5), 'rating_code'] = 0
print(all_d.shape)
all_d.tail(5)

(1562618, 11)


,user_id,item_id,progress,rating,start_date,age,sex,year,authors_code,auhurs_genres,rating_code
1562613,43983,96797.0,100.0,5.0,2019-12-31,65_inf,0,2019,1,любовное фэнтези елизавета соболянская,1.0
1562614,63445,157812.0,5.0,0.0,2019-12-31,25_34,0,2015,1,зарубежные детективы триллеры современные дете...,1.0
1562615,104572,242176.0,1.0,0.0,2019-12-31,55_64,0,2015,1,современная русская литература гузель яхина,1.0
8501,10309,NaN,NaN,NaN,NaT,18_24,0,5000,0,nan noauthors,1.0
129118,156235,NaN,NaN,NaN,NaT,25_34,1,5000,0,nan noauthors,1.0


In [ ]:
all_d[(all_d['user_id']==10309) | (all_d['user_id']==156235)]

,user_id,item_id,progress,rating,start_date,age,sex,year,authors_code,auhurs_genres,rating_code
8501,10309,NaN,NaN,NaN,NaT,18_24,0,5000,0,nan noauthors,1.0
129118,156235,NaN,NaN,NaN,NaT,25_34,1,5000,0,nan noauthors,1.0


In [ ]:
#найдем самые часто стречающиеся книги для соответвующего возраста и пола и заменим их
impl_0 = all_d.loc[(all_d['age']=='18_24') & (all_d['sex']==0) & (all_d['rating'] >= 4.5), 'item_id'].value_counts().index[0]
impl_1 = all_d.loc[(all_d['age']=='25_34') & (all_d['sex']==1) & (all_d['rating'] >= 4.5), 'item_id'].value_counts().index[0]
all_d.loc[all_d['user_id']==10309, 'item_id'] = impl_0
all_d.loc[all_d['user_id']==156235, 'item_id'] = impl_1
all_d.tail(3)

,user_id,item_id,progress,rating,start_date,age,sex,year,authors_code,auhurs_genres,rating_code
1562615,104572,242176.0,1.0,0.0,2019-12-31,55_64,0,2015,1,современная русская литература гузель яхина,1.0
8501,10309,186002.0,NaN,NaN,NaT,18_24,0,5000,0,nan noauthors,1.0
129118,156235,112869.0,NaN,NaN,NaT,25_34,1,5000,0,nan noauthors,1.0


In [ ]:
items_inv_mapping = dict(enumerate(all_d['item_id'].unique()))
items_mapping = {v: k for k, v in items_inv_mapping.items()}

users_inv_mapping = dict(enumerate(all_d['user_id'].unique()))
users_mapping = {v: k for k, v in users_inv_mapping.items()}

item_titles = pd.Series(df_items['title'].values, index=df_items['id']).to_dict()

In [ ]:
# Оценим книги от одного до двух каждым пользователем где каждая последующая книга с которой было взаимодействие оценивается больше предыдущего. 
# Даты у нас последовательно выстроены, поэтому не смотрим сортировку
list_id = all_d.user_id.unique().tolist()
next_book = {}
for i in list_id:
    time_list = all_d.loc[all_d.user_id == i]['start_date'].index
    ind = 0
    for j in time_list:
        next_book[j] = 1 + (ind/(len(time_list)))  
        ind += 1
all_d = all_d.join(pd.DataFrame(pd.Series(next_book)))
all_d = all_d.rename({0: 'book_rat'}, axis = 1)


In [ ]:
Data_all = pd.merge(all_d, df[['user_id', 'item_id', 'start_date']], how='left', on=['user_id', 'item_id'])
Data_all.tail(3)

In [ ]:
# Ну и перестроим таблицу дат по количеству дней оставшихся до конечной десяти дней после последней записи нашей, 
# ну по количеству прогнозируемых книг
all_d['start_date'] = (datetime.datetime(2020, 1, 10) - all_d['start_date']).dt.days
all_d['start_date'] = all_d['start_date'].fillna(10)
all_d.rename({'start_date': 'days'}, axis = 1, inplace=True)


# Добавим каждому пользователю количество прочитанных ими книг
user_book = pd.DataFrame(all_d.groupby('user_id').count()['item_id']).rename({'item_id': 'user_book'}, axis = 1).reset_index()
all_d = all_d.merge(user_book, how = 'left')

# Добавим количество взаимодействий с каждой книгой всеми пользователями
count_book = pd.DataFrame(all_d.groupby('item_id').count()['user_id']).rename({'user_id': 'count_book'}, axis = 1).reset_index()
all_d = all_d.merge(count_book, how = 'left')
all_d['count_book'] = all_d['count_book'].fillna(0)
all_d['count_book'] = all_d['count_book'].astype('int16')

print(all_d.shape)        
all_d.tail(3)

(1562618, 15)


,user_id,item_id,progress,rating,days,age,sex,year,authors_code,auhurs_genres,rating_code,book_rat,user_book,count_book,start_date
1562615,43983,96797.0,100.0,5.0,10.0,65_inf,0,2019,1,любовное фэнтези елизавета соболянская,1.0,1.992754,138,13,2019-12-31
1562616,63445,157812.0,5.0,0.0,10.0,25_34,0,2015,1,зарубежные детективы триллеры современные дете...,1.0,1.928571,14,18,2019-12-31
1562617,104572,242176.0,1.0,0.0,10.0,55_64,0,2015,1,современная русская литература гузель яхина,1.0,1.833333,6,149,2019-12-31


In [ ]:
#Приступим к кодированию категориальных признаков и исправим формат кодирования авторов
all_data_code = all_d.copy(deep = True)
encod_age = LabelEncoder()
all_data_code.age = encod_age.fit_transform(all_data_code.age)

encod_sex = LabelEncoder()
all_data_code.sex.replace(0, 'f', inplace = True)
all_data_code.sex.replace(1, 'm', inplace = True)
all_data_code.sex = encod_sex.fit_transform(all_data_code.sex)

encod_year = LabelEncoder()
all_data_code.year = encod_year.fit_transform(all_data_code.year)

print(all_data_code.shape)
all_data_code.tail(3)

(1562618, 15)


,user_id,item_id,progress,rating,days,age,sex,year,authors_code,auhurs_genres,rating_code,book_rat,user_book,count_book,start_date
1562615,43983,96797.0,100.0,5.0,10.0,5,0,20,1,любовное фэнтези елизавета соболянская,1.0,1.992754,138,13,2019-12-31
1562616,63445,157812.0,5.0,0.0,10.0,1,0,16,1,зарубежные детективы триллеры современные дете...,1.0,1.928571,14,18,2019-12-31
1562617,104572,242176.0,1.0,0.0,10.0,4,0,16,1,современная русская литература гузель яхина,1.0,1.833333,6,149,2019-12-31


In [ ]:
#Теперь изменим целевой признак составив одну функцию из рейтинга прогресса и выведенных нами функций
all_data_code['metric'] = (all_data_code['progress'] / 100) + 1
all_data_code['metric'] = (all_data_code['rating'] + 1).multiply(all_data_code['book_rat'], 
                                    axis = 'rows').multiply(all_data_code['metric'], axis = 'rows')
print(all_data_code.shape)

(1562618, 16)


In [ ]:
all_data_code = all_data_code[['user_id', 'item_id', 'metric', 'rating', 'rating_code', 'book_rat', 'progress', 'days', 'age', 'sex', 'user_book',
       'year', 'authors_code', 'count_book', 'auhurs_genres']]
all_data_code['start_date'] = Data_all['start_date']
all_data_code.tail(3)

,user_id,item_id,metric,rating,rating_code,book_rat,progress,days,age,sex,user_book,year,authors_code,count_book,auhurs_genres,start_date
1562615,43983,96797.0,23.913043,5.0,1.0,1.992754,100.0,10.0,5,0,138,20,1,13,любовное фэнтези елизавета соболянская,2019-12-31
1562616,63445,157812.0,2.025000,0.0,1.0,1.928571,5.0,10.0,1,0,14,16,1,18,зарубежные детективы триллеры современные дете...,2019-12-31
1562617,104572,242176.0,1.851667,0.0,1.0,1.833333,1.0,10.0,4,0,6,16,1,149,современная русская литература гузель яхина,2019-12-31


#Check_point

In [ ]:
'''
all_data_code.to_pickle('/content/drive/MyDrive/Colab Notebooks/MTS_teta/Kaggle_recomend_sys/all_data_mts.pkl')
'''
all_data_code = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/MTS_teta/Kaggle_recomend_sys/all_data_mts.pkl')
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MTS_teta/Kaggle_recomend_sys/sample_submission.csv')
df_items = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/MTS_teta/Kaggle_recomend_sys/items_preprocessed.pickle')

In [ ]:
items_inv_mapping = dict(enumerate(all_data_code['item_id'].unique()))
items_mapping = {v: k for k, v in items_inv_mapping.items()}

users_inv_mapping = dict(enumerate(all_data_code['user_id'].unique()))
users_mapping = {v: k for k, v in users_inv_mapping.items()}

item_titles = pd.Series(df_items['title'].values, index=df_items['id']).to_dict()

#Validation
Для наших данных выбрем 7 последних дней и будем тестировать на них последовательно (1 test fold - 1 день).

Но теперь нам нужно учитывать проблему холодного старта. Это основная проблем классических методов над матрицей взаимодействий. Поэтому напишем свой класс для разбиения исходного датафрейма на train/test.

In [ ]:
class TimeRangeSplit():


    def __init__(self, 
                 start_date, 
                 end_date=None, 
                 freq='D', 
                 periods=None, 
                 tz=None, 
                 normalize=False, 
                 closed=None, 
                 train_min_date=None,
                 filter_cold_users=True, 
                 filter_cold_items=True, 
                 filter_already_seen=True):
        
        self.start_date = start_date
        if end_date is None and periods is None:
            raise ValueError("Either 'end_date' or 'periods' must be non-zero, not both at the same time.")

        self.end_date = end_date
        self.freq = freq
        self.periods = periods
        self.tz = tz
        self.normalize = normalize
        self.closed = closed
        self.train_min_date = pd.to_datetime(train_min_date, errors='raise')
        self.filter_cold_users = filter_cold_users
        self.filter_cold_items = filter_cold_items
        self.filter_already_seen = filter_already_seen
        # диапозон дат
        self.date_range = pd.date_range(
            start=start_date, 
            end=end_date, 
            freq=freq, 
            periods=periods, 
            tz=tz, 
            normalize=normalize, 
            closed=closed)
        # максимально возможно число разбиений
        self.max_n_splits = max(0, len(self.date_range) - 1)
        if self.max_n_splits == 0:
            raise ValueError("Provided parametrs set an empty date range.") 


    def split(self, 
              df, 
              user_column='user_id',
              item_column='item_id',
              datetime_column='date',
              fold_stats=False):
        '''
          метод, который возвращает генератор индексов для одного фолда валидации
        '''
        df_datetime = df[datetime_column]
        if self.train_min_date is not None:
            train_min_mask = df_datetime >= self.train_min_date
        else:
            train_min_mask = df_datetime.notnull()

        date_range = self.date_range[(self.date_range >= df_datetime.min()) & 
                                     (self.date_range <= df_datetime.max())]

        for start, end in pairwise(date_range):
            fold_info = {
                'Start date': start,
                'End date': end
            }
            train_mask = train_min_mask & (df_datetime < start)
            train_idx = df.index[train_mask]
            if fold_stats:
                fold_info['Train'] = len(train_idx)

            test_mask = (df_datetime >= start) & (df_datetime < end)
            test_idx = df.index[test_mask]
            
            if self.filter_cold_users:
                new = np.setdiff1d(
                    df.loc[test_idx, user_column].unique(), 
                    df.loc[train_idx, user_column].unique())
                new_idx = df.index[test_mask & df[user_column].isin(new)]
                test_idx = np.setdiff1d(test_idx, new_idx)
                test_mask = df.index.isin(test_idx)
                if fold_stats:
                    fold_info['New users'] = len(new)
                    fold_info['New users interactions'] = len(new_idx)

            if self.filter_cold_items:
                new = np.setdiff1d(
                    df.loc[test_idx, item_column].unique(), 
                    df.loc[train_idx, item_column].unique())
                new_idx = df.index[test_mask & df[item_column].isin(new)]
                test_idx = np.setdiff1d(test_idx, new_idx)
                test_mask = df.index.isin(test_idx)
                if fold_stats:
                    fold_info['New items'] = len(new)
                    fold_info['New items interactions'] = len(new_idx)

            if self.filter_already_seen:
                user_item = [user_column, item_column]
                train_pairs = df.loc[train_idx, user_item].set_index(user_item).index
                test_pairs = df.loc[test_idx, user_item].set_index(user_item).index
                intersection = train_pairs.intersection(test_pairs)
                test_idx = test_idx[~test_pairs.isin(intersection)]
                # test_mask = rd.df.index.isin(test_idx)
                if fold_stats:
                    fold_info['Known interactions'] = len(intersection)

            if fold_stats:
                fold_info['Test'] = len(test_idx)

            yield (train_idx, test_idx, fold_info)


    def get_n_splits(self, df, datetime_column='date'):
        '''
          число разбиений, которое получается по факту из данных
        '''
        df_datetime = df[datetime_column]
        if self.train_min_date is not None:
            df_datetime = df_datetime[df_datetime >= self.train_min_date]

        date_range = self.date_range[(self.date_range >= df_datetime.min()) & 
                                     (self.date_range <= df_datetime.max())]

        return max(0, len(date_range) - 1)

In [ ]:
last_date = all_data_code['start_date'].max().normalize()
folds = 7
start_date = last_date - pd.Timedelta(days=folds)
start_date, last_date

(Timestamp('2019-12-24 00:00:00'), Timestamp('2019-12-31 00:00:00'))

In [ ]:
cv = TimeRangeSplit(start_date=start_date, periods=folds+1)

cv.max_n_splits, cv.get_n_splits(all_data_code, datetime_column='start_date')

(7, 7)

In [ ]:
cv.date_range

DatetimeIndex(['2019-12-24', '2019-12-25', '2019-12-26', '2019-12-27',
               '2019-12-28', '2019-12-29', '2019-12-30', '2019-12-31'],
              dtype='datetime64[ns]', freq='D')

In [ ]:
folds_with_stats = list(cv.split(
    all_data_code, 
    user_column='user_id',
    item_column='item_id',
    datetime_column='start_date',
    fold_stats=True
))

folds_info_with_stats = pd.DataFrame([info for _, _, info in folds_with_stats])

In [ ]:
folds_info_with_stats

,Start date,End date,Train,New users,New users interactions,New items,New items interactions,Known interactions,Test
0,2019-12-24,2019-12-25,1545838,3,3,0,0,0,2076
1,2019-12-25,2019-12-26,1547917,5,5,0,0,0,2102
2,2019-12-26,2019-12-27,1550024,3,5,0,0,0,2120
3,2019-12-27,2019-12-28,1552149,2,3,0,0,0,2096
4,2019-12-28,2019-12-29,1554248,2,2,0,0,0,2103
5,2019-12-29,2019-12-30,1556353,3,3,0,0,0,2051
6,2019-12-30,2019-12-31,1558407,3,3,0,0,0,2132


In [ ]:
def compute_metrics(df_true, df_pred, top_N=10):
    result = {}
    test_recs = df_true.set_index(['user_id', 'item_id']).join(df_pred.set_index(['user_id', 'item_id']))
    test_recs = test_recs.sort_values(by=['user_id', 'rank'])

    test_recs['users_item_count'] = test_recs.groupby(level='user_id')['rank'].transform(np.size)
    test_recs['reciprocal_rank'] = (1 / test_recs['rank']).fillna(0)
    test_recs['cumulative_rank'] = test_recs.groupby(level='user_id').cumcount() + 1
    test_recs['cumulative_rank'] = test_recs['cumulative_rank'] / test_recs['rank']
    
    users_count = test_recs.index.get_level_values('user_id').nunique()
    for k in range(1, top_N + 1):
        hit_k = f'hit@{k}'
        test_recs[hit_k] = test_recs['rank'] <= k
        result[f'Precision@{k}'] = (test_recs[hit_k] / k).sum() / users_count
        result[f'Recall@{k}'] = (test_recs[hit_k] / test_recs['users_item_count']).sum() / users_count

    result[f'MAP@{top_N}'] = (test_recs["cumulative_rank"] / test_recs["users_item_count"]).sum() / users_count
    result[f'MRR'] = test_recs.groupby(level='user_id')['reciprocal_rank'].max().mean()
    return pd.Series(result)

#Pop model

Из baseline

In [ ]:
class Usy_PopularRecommender():
    def __init__(self, max_K=100, days=30, item_column='item_id', dt_column='date'):
        self.max_K = max_K
        self.days = days
        self.item_column = item_column
        self.dt_column = dt_column
        self.recommendations = []
        
    def fit(self, df):
        min_date = df[self.dt_column].max().normalize() - pd.DateOffset(days=self.days)
        self.recommendations = df.loc[df[self.dt_column] > min_date, 
                                      self.item_column].value_counts().head(self.max_K).index.values
    
    def recommend(self, users=None, N=10):
        recs = self.recommendations[:N]
        if users is None:
            return recs
        else:
            return list(islice(cycle([recs]), len(users)))

In [ ]:
map10 = []
mrr = []
for i in tqdm(range(len(folds_with_stats))):
    train_idx, test_idx, info = folds_with_stats[i]
    train = all_d.loc[train_idx]
    test = all_d.loc[test_idx]
    
    pop = Usy_PopularRecommender(days=30, dt_column='start_date')
    pop.fit(train)

    test_pred_pop = pd.DataFrame({'user_id': test['user_id'].unique()})

    # сделаем предикт моделью популярного сразу для всех id
    test_pred_pop['item_id'] = pop.recommend(test_pred_pop['user_id'], N=10)
    recs = test_pred_pop.explode('item_id')
    recs['rank'] = recs.groupby('user_id').cumcount() + 1
    fold_result = compute_metrics(test, recs, top_N)
    map10 += [fold_result['MAP@10']]
    mrr += [fold_result['MRR']]

pop_valid_metrics = pd.DataFrame({'MAP@10': map10, 'MRR': mrr})
print(f"mean map10: {pop_valid_metrics['MAP@10'].mean()}\nmean mrr: {pop_valid_metrics['MRR'].mean()}")


mean map10: 0.012696768828078455
mean mrr: 0.014356964517941991


Выбор популярных из некоторых групп

In [ ]:
class Reiting_PopularRecommender():
    def __init__(self, max_K=100, days=30, item_column='item_id', dt_column='date'):
        self.max_K = max_K
        self.days = days
        self.item_column = item_column
        self.dt_column = dt_column
        self.recommendations = []
        
    def fit(self, df):
        min_date = df[self.dt_column].max().normalize() - pd.DateOffset(days=self.days)
        self.recommendations = df.loc[(df[self.dt_column] > min_date) &
                                      (df['rating'] >= 3), 
                                      self.item_column].value_counts().head(self.max_K).index.values
    
    def recommend(self, users=None, N=10):
        recs = self.recommendations[:N]
        if users is None:
            return recs
        else:
            return list(islice(cycle([recs]), len(users)))

In [ ]:
class Sex_PopularRecommender():

  
    def __init__(self, max_K=100, days=30, item_column='item_id', dt_column='date', age_recs=False):
        self.max_K = max_K
        self.days = days
        self.item_column = item_column
        self.dt_column = dt_column
        self.recommendations = []
        self.age_recs = age_recs
        
    def fit(self, df, df_users=None):
        min_date = df[self.dt_column].max().normalize() - pd.DateOffset(days=self.days)

        if self.age_recs:
            self.recommendations = {}
            self.df = pd.merge(df, df_users, on='user_id', how='left')

            self.sex_group = self.df['sex'].unique()
            for age in self.age_group:
                self.recommendations[sex] = self.df.loc[(self.df['sex'] == sex) &
                                                        (self.df[self.dt_column] > min_date), 
                                                        'item_id'].value_counts().head(100).index.values
            # мало ли не найдется никакой группы
            self.recommendations['None_info'] = df.loc[df[self.dt_column] > min_date, 
                                                       self.item_column].value_counts().head(self.max_K).index.values
        else:
            self.recommendations = df.loc[df[self.dt_column] > min_date, 
                                          self.item_column].value_counts().head(self.max_K).index.values
    
    def recommend(self, users=None, N=10):

        if self.age_recs:
            recs = []
            for user in users:
                user_sex = self.df[self.df['user_id'] == user]['sex'].unique()
                if user_sex in self.sex_group:
                    recs += [list(self.recommendations[str(user_sex[0])][:N].astype(int))]
                else:
                    recs += [list(self.recommendations['None_info'][:N].astype(int))]
            return recs

        else:
            recs = self.recommendations[:N].astype(int)
            if users is None:
                return recs
            else:
                return list(islice(cycle([recs]), len(users)))

In [ ]:
map10 = []
mrr = []
for i in tqdm(range(len(folds_with_stats))):
    train_idx, test_idx, info = folds_with_stats[i]
    train = all_d.loc[train_idx]
    test = all_d.loc[test_idx]
    
    pop = Sex_PopularRecommender(days=30, dt_column='start_date')
    pop.fit(train)

    test_pred_pop = pd.DataFrame({'user_id': test['user_id'].unique()})

    test_pred_pop['item_id'] = pop.recommend(test_pred_pop['user_id'], N=10)
    recs = test_pred_pop.explode('item_id')
    recs['rank'] = recs.groupby('user_id').cumcount() + 1
    fold_result = compute_metrics(test, recs, top_N)
    map10 += [fold_result['MAP@10']]
    mrr += [fold_result['MRR']]

sex_pop_valid_metrics = pd.DataFrame({'MAP@10': map10, 'MRR': mrr})
print(f"mean map10: {sex_pop_valid_metrics['MAP@10'].mean()}\nmean mrr: {sex_pop_valid_metrics['MRR'].mean()}")


mean map10: 0.012696768828078455
mean mrr: 0.014356964517941991


In [ ]:
# 30 -- 0.01269

#Implicit model

In [ ]:
def get_coo_matrix(df, 
                   user_col='user_id', 
                   item_col='item_id', 
                   weight_col='metric', 
                   users_mapping=users_mapping, 
                   items_mapping=items_mapping):
    if weight_col is None:
        weights = np.ones(len(df), dtype=np.float32)
    else:
        weights = df[weight_col].astype(np.float32) 

    interaction_matrix = sp.coo_matrix((
        weights, 
        (
            df[user_col].map(users_mapping.get), 
            df[item_col].map(items_mapping.get)
        )
    ))
    return interaction_matrix

In [ ]:
def generate_implicit_recs_mapper(model, train_matrix, N, user_mapping, item_inv_mapping):
    def _recs_mapper(user):
        user_id = user_mapping[user]
        recs = model.recommend(user_id, 
                               train_matrix, 
                               N=N, 
                               filter_already_liked_items=True)
        return [int(item_inv_mapping[item]) for item, _ in recs]
    return _recs_mapper

#ALS

In [ ]:
map10 = []
mrr = []
for train_idx, test_idx, info in tqdm(folds_with_stats):

    train = all_data_code.loc[train_idx]
    test = all_data_code.loc[test_idx]
    train_mat = get_coo_matrix(train).tocsr()
    
    als_model = AlternatingLeastSquares(factors=8, regularization=0.0, iterations=20)
    als_model.fit(train_mat.T) 
    
    mapper = generate_implicit_recs_mapper(als_model, train_mat, 10, users_mapping, items_inv_mapping)
    
    recs = pd.DataFrame({
        'user_id': test['user_id'].unique()
    })
    recs['item_id'] = recs['user_id'].map(mapper)
    recs = recs.explode('item_id')
    recs['rank'] = recs.groupby('user_id').cumcount() + 1
    metrics = compute_metrics(test, recs)
    
    fold_result = pd.Series(metrics)
    map10 += [fold_result['MAP@10']]
    mrr += [fold_result['MRR']]

als_valid_metrics = pd.DataFrame({'MAP@10': map10, 'MRR': mrr})
print(f"mean map10: {als_valid_metrics['MAP@10'].mean()}\nmean mrr: {als_valid_metrics['MRR'].mean()}")



mean map10: 0.00012838072932375784
mean mrr: 0.00017396324935930882


#Item to Item

In [ ]:
K = 10
II_valid_metrics = {}
for model_class in [CosineRecommender, BM25Recommender, TFIDFRecommender]:
    map10 = []
    mrr = []
    for train_idx, test_idx, info in tqdm(folds_with_stats):
        train = df.loc[train_idx]
        test = df.loc[test_idx]
        
        train_mat = get_coo_matrix(train).tocsr()
        
        model = model_class(K=K)
        model.fit(train_mat.T, show_progress=False) 

        mapper = generate_implicit_recs_mapper(model, train_mat, top_N, users_mapping, items_inv_mapping)

        recs = pd.DataFrame({
            'user_id': test['user_id'].unique()
        })
        recs['item_id'] = recs['user_id'].map(mapper)
        recs = recs.explode('item_id')
        recs['rank'] = recs.groupby('user_id').cumcount() + 1
        fold_result = compute_metrics(test, recs, top_N)
        fold_result = pd.Series(metrics)
        map10 += [fold_result['MAP@10']]
        mrr += [fold_result['MRR']]

    II_valid_metrics[model.__class__.__name__] = pd.DataFrame({'MAP@10': map10, 'MRR': mrr})
    print(f"========{model.__class__.__name__}=======\
            \nmean map10: {II_valid_metrics[model.__class__.__name__]['MAP@10'].mean()}\
            \nmean mrr: {II_valid_metrics[model.__class__.__name__]['MRR'].mean()}")

In [ ]:
K = 10
validation_results = pd.DataFrame()
for model_class in [CosineRecommender, BM25Recommender, TFIDFRecommender]:
    for train_idx, test_idx, info in tqdm(folds_with_stats):
        print(f"test range - from {info['Start date']} to {info['End date']}")
        train = all_data_code.loc[train_idx]
        test = all_data_code.loc[test_idx]
        print(f'train shape - {train.shape}, test shape - {test.shape}')
        
        train_mat = get_coo_matrix(train).tocsr()
        
        model = model_class(K=K)
        model.fit(train_mat.T, show_progress=False) 

        mapper = generate_implicit_recs_mapper(model, train_mat, top_N, users_mapping, items_inv_mapping)

        recs = pd.DataFrame({
            'user_id': test['user_id'].unique()
        })
        recs['item_id'] = recs['user_id'].map(mapper)
        recs = recs.explode('item_id')
        recs['rank'] = recs.groupby('user_id').cumcount() + 1
        fold_result = compute_metrics(test, recs, top_N)
        fold_result.at['Date'] = info['Start date']
        fold_result.at['argv'] = f'K={K}'
        fold_result.at['Model'] = model.__class__.__name__

        validation_results = validation_results.append(fold_result, ignore_index=True)

test range - from 2019-12-24 00:00:00 to 2019-12-25 00:00:00
train shape - (1545838, 16), test shape - (2076, 16)
test range - from 2019-12-25 00:00:00 to 2019-12-26 00:00:00
train shape - (1547917, 16), test shape - (2102, 16)
test range - from 2019-12-26 00:00:00 to 2019-12-27 00:00:00
train shape - (1550024, 16), test shape - (2120, 16)
test range - from 2019-12-27 00:00:00 to 2019-12-28 00:00:00
train shape - (1552149, 16), test shape - (2096, 16)
test range - from 2019-12-28 00:00:00 to 2019-12-29 00:00:00
train shape - (1554248, 16), test shape - (2103, 16)
test range - from 2019-12-29 00:00:00 to 2019-12-30 00:00:00
train shape - (1556353, 16), test shape - (2051, 16)
test range - from 2019-12-30 00:00:00 to 2019-12-31 00:00:00
train shape - (1558407, 16), test shape - (2132, 16)



test range - from 2019-12-24 00:00:00 to 2019-12-25 00:00:00
train shape - (1545838, 16), test shape - (2076, 16)
test range - from 2019-12-25 00:00:00 to 2019-12-26 00:00:00
train shape - (1547917, 16), test shape - (2102, 16)
test range - from 2019-12-26 00:00:00 to 2019-12-27 00:00:00
train shape - (1550024, 16), test shape - (2120, 16)
test range - from 2019-12-27 00:00:00 to 2019-12-28 00:00:00
train shape - (1552149, 16), test shape - (2096, 16)
test range - from 2019-12-28 00:00:00 to 2019-12-29 00:00:00
train shape - (1554248, 16), test shape - (2103, 16)
test range - from 2019-12-29 00:00:00 to 2019-12-30 00:00:00
train shape - (1556353, 16), test shape - (2051, 16)
test range - from 2019-12-30 00:00:00 to 2019-12-31 00:00:00
train shape - (1558407, 16), test shape - (2132, 16)



test range - from 2019-12-24 00:00:00 to 2019-12-25 00:00:00
train shape - (1545838, 16), test shape - (2076, 16)
test range - from 2019-12-25 00:00:00 to 2019-12-26 00:00:00
train shape - (1547917, 16), test shape - (2102, 16)
test range - from 2019-12-26 00:00:00 to 2019-12-27 00:00:00
train shape - (1550024, 16), test shape - (2120, 16)
test range - from 2019-12-27 00:00:00 to 2019-12-28 00:00:00
train shape - (1552149, 16), test shape - (2096, 16)
test range - from 2019-12-28 00:00:00 to 2019-12-29 00:00:00
train shape - (1554248, 16), test shape - (2103, 16)
test range - from 2019-12-29 00:00:00 to 2019-12-30 00:00:00
train shape - (1556353, 16), test shape - (2051, 16)
test range - from 2019-12-30 00:00:00 to 2019-12-31 00:00:00
train shape - (1558407, 16), test shape - (2132, 16)



In [ ]:
validation_results.groupby(['Model', 'argv']).agg({
    'MRR': ['mean', 'std', 'min', 'max'],
    'MAP@10': ['mean', 'std', 'min', 'max'],
    'Recall@10': ['mean', 'std', 'min', 'max'],
})

MRR            ... Recall@10          
                            mean       std  ...       min       max
Model             argv                      ...                    
BM25Recommender   K=10  0.020451  0.005115  ...  0.030599  0.047078
CosineRecommender K=10  0.022424  0.004645  ...  0.035059  0.050584
TFIDFRecommender  K=10  0.022351  0.004534  ...  0.034614  0.050946

[3 rows x 12 columns]

In [ ]:
all_train_mat = get_coo_matrix(all_data_code).tocsr()
        
II_model = model_class(K=10)
II_model.fit(all_train_mat.T, show_progress=True) 

In [ ]:
mapper = generate_implicit_recs_mapper(II_model, all_train_mat, top_N, users_mapping, items_inv_mapping)

recs = pd.DataFrame({
            'user_id': submission['Id'].unique()
})
recs['item_id'] = recs['user_id'].map(mapper)
recs['item_id'] = recs['item_id']

In [ ]:
recs.head()

,user_id,item_id
0,10001,"[296208, 297168, 235765, 237354, 266602, 13279..."
1,10002,"[378939, 43309, 327688, 54436, 240408, 12597, ..."
2,100152,"[167562, 137774, 168056, 44107, 202167, 113057..."
3,100197,"[289973, 179505, 316246, 364778, 160735, 25343..."
4,100284,"[343815, 168733, 273724, 102108, 164263, 31257..."


In [ ]:
pred = []
ids = []
for id, recomm in tqdm(recs.groupby('user_id')):
    ids += [id]
    pred += [' '.join(map(str, recomm['item_id'].values[0]))]
submission_II = pd.DataFrame({'Id': ids,
                              'Predicted': pred})
submission_II.head()

,Id,Predicted
0,24,141397 205911 298589 245698 152735 118323 2913...
1,41,83794 224145 196301 85586 273416 389662 23059 ...
2,67,170920 172266 230724 347816 153310 233179 2223...
3,101,345611 233005 260144 194135 178241 122405 3686...
4,107,273156 371497 33517 60369 116799 228431 8073 3...


In [ ]:
submission_II.to_csv('submission_II.csv', index=False)

In [ ]:
def make_submission(df, id_col='user_id', predicted_col='item_id'):
    
    df_copy = df.copy()
    df_copy.loc[:, id_col] = df_copy[id_col].astype(str)
    df_copy.loc[:, predicted_col] = df_copy[predicted_col].astype(str)
    df_copy = df_copy.groupby(id_col, as_index=False).agg({predicted_col: lambda x: ' '.join(list(x))})
    df_copy.rename(columns={id_col: 'Id', predicted_col: 'Predicted'}, inplace=True)
    
    return df_copy[['Id', 'Predicted']]

In [ ]:
submission_II_1 = make_submission(recs)
submission_II_1.head()

,Id,Predicted
0,10001,"[296208, 297168, 235765, 237354, 266602, 13279..."
1,10002,"[378939, 43309, 327688, 54436, 240408, 12597, ..."
2,100152,"[167562, 137774, 168056, 44107, 202167, 113057..."
3,100197,"[289973, 179505, 316246, 364778, 160735, 25343..."
4,100284,"[343815, 168733, 273724, 102108, 164263, 31257..."


##User to User

#Pop + Item_to_Item

In [ ]:
def generate_POP_II_recs_mapper(model_II, model_POP, train_matrix, train_df, N, user_mapping, item_inv_mapping, small_info=5):
    def _recs_mapper(user):
        if train_df[train_df['user_id']==user].shape[0] < small_info:
            recs = model_POP.recommend(N=N)
            return [int(item) for item in recs]
        else:
            user_id = user_mapping[user]
            recs = model_II.recommend(user_id, 
                                      train_matrix, 
                                      N=N, 
                                      filter_already_liked_items=True)
            return [int(item_inv_mapping[item]) for item, _ in recs]
    return _recs_mapper

In [ ]:
K = 10
small_info = 10
for train_idx, test_idx, info in tqdm(folds_with_stats):
    train = all_data_code.loc[train_idx]
    test = all_data_code.loc[test_idx]

    train_mat = get_coo_matrix(train).tocsr()

    model_II = CosineRecommender(K=K)
    model_II.fit(train_mat.T, show_progress=False)    

    model_POP = Usy_PopularRecommender(days=30, dt_column='start_date')
    model_POP.fit(train)

    mapper = generate_POP_II_recs_mapper(model_II, model_POP, train_mat, train, top_N, users_mapping, items_inv_mapping, small_info=small_info)   
    recs = pd.DataFrame({
        'user_id': test['user_id'].unique()
    })
    recs['item_id'] = recs['user_id'].map(mapper)
    recs = recs.explode('item_id')
    recs['rank'] = recs.groupby('user_id').cumcount() + 1
    fold_result = compute_metrics(test, recs, top_N)
    fold_result.at['Date'] = info['Start date']
    fold_result.at['argv'] = f'K={K}'
    fold_result.at['Model'] = f'POP+II_{small_info}'
    validation_results = validation_results.append(fold_result, ignore_index=True)

In [ ]:
validation_results.groupby(['Model', 'argv']).agg({
    'MRR': ['mean', 'std', 'min', 'max'],
    'MAP@10': ['mean', 'std', 'min', 'max'],
    'Recall@10': ['mean', 'std', 'min', 'max'],
})

MRR            ... Recall@10          
                                mean       std  ...       min       max
Model                 argv                      ...                    
POP+II                K=10  0.028011  0.005461  ...  0.040960  0.062386
POP+II_10             K=10  0.031166  0.005191  ...  0.047844  0.074342
POP+II_11             K=10  0.030724  0.005215  ...  0.046984  0.073731
POP+II_12             K=10  0.030711  0.005285  ...  0.046287  0.073731
POP+II_4              K=10  0.025967  0.005410  ...  0.038665  0.057963
POP+II_6              K=10  0.029766  0.005355  ...  0.043255  0.064975
POP+II_7              K=10  0.030455  0.005311  ...  0.045263  0.067833
POP+II_8              K=10  0.030865  0.005321  ...  0.046984  0.070746
POP+II_9              K=10  0.031145  0.005489  ...  0.046984  0.073426
POP+II_BM25P10        K=10  0.028310  0.005305  ...  0.041930  0.069732
POP+II_BM25P_noweig10 K=10  0.026240  0.003519  ...  0.040304  0.065293

[11 rows x 12 columns]

In [ ]:
small_info = 10
K = 10

all_train_mat = get_coo_matrix(all_data_code).tocsr()
        
model_II = CosineRecommender(K=K)
model_II.fit(all_train_mat.T, show_progress=True) 

model_POP = Usy_PopularRecommender(days=30, dt_column='start_date')
model_POP.fit(all_data_code)

mapper = generate_POP_II_recs_mapper(model_II, model_POP, train_mat, train, top_N, users_mapping, items_inv_mapping, small_info=small_info)

recs = pd.DataFrame({
            'user_id': submission['Id'].unique()
})
recs['item_id'] = recs['user_id'].map(mapper)
recs['item_id'] = recs['item_id']
recs.head(3)

,user_id,item_id
0,10001,"[283713, 276903, 184549, 168963, 357309, 14317..."
1,10002,"[283713, 276903, 184549, 168963, 357309, 14317..."
2,100152,"[167562, 137774, 168056, 217643, 202167, 44107..."


In [ ]:
pred = []
ids = []
for id, recomm in tqdm(recs.groupby('user_id')):
    ids += [id]
    pred += [' '.join(map(str, recomm['item_id'].values[0]))]
submission_POP_II = pd.DataFrame({'Id': ids,
                                  'Predicted': pred})
submission_POP_II.head()

,Id,Predicted
0,24,283713 276903 184549 168963 357309 143175 3852...
1,41,283713 276903 184549 168963 357309 143175 3852...
2,67,283713 276903 184549 168963 357309 143175 3852...
3,101,283713 276903 184549 168963 357309 143175 3852...
4,107,283713 276903 184549 168963 357309 143175 3852...


In [ ]:
submission_POP_II.to_csv('submission_POP_II.csv', index=False)